In [121]:
import json
import requests
from collections import defaultdict
from time import sleep
from bs4 import BeautifulSoup
import pandas as pd

In [115]:
# scrape week by week results (from www.statto.com)
# and save as json files by season

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64)'}
base_url = 'http://www.statto.com/football/stats/england/premier-league/'
years = range(1995, 2015)
all_data = []


def get_dates(url):
    soup = BeautifulSoup(requests.get(url, headers=headers).text)
    table = soup.find('table', {'class': 'league-table'})
    date_select = table.find('select', {'id': 'dates'})
    dates = [d['value'] for d in date_select.findAll('option')]
    return dates


def get_standings(url):
    soup = BeautifulSoup(requests.get(url, headers=headers).text)
    tbody = soup.find('table', {'class': 'league-table'}).find('tbody')
    rows = tbody.findAll('tr')

    data = []
    for row in rows:
        cells = row.findAll('td')
        data.append({
            'name': cells[1].text,
            'games_played': cells[2].text,
            'goals_total': cells[6].text,
            'goals_diff': cells[-4].text,
            'points': cells[-3].text,
        })
    return data
        

def save_data(data, name):
    fname = '../data/{}.json'.format(name)
    with open(fname, 'w') as f:
        json.dump(data, f)
    

for year in years:
    sleep(5)
    season = '{}-{}'.format(year, year + 1)
    print 'fetching {} season data...'.format(season)
    
    season_url = '{}/{}/table/'.format(base_url, season)
    dates = get_dates(season_url)
    print '{} dates'.format(len(dates))
    
    year_data = []
    for date in dates:
        sleep(1)
        date = '' if date == '-' else date # clunky
        week_url = season_url + date
        
        try:
            standings = get_standings(week_url)
        except Exception:
            print 'trouble parsing {}'.format(week_url)
            continue
            
        year_data.append({
            'season': season,
            'as_of_date': date,
            'url': week_url,
            'standings': standings,
        })
    save_data(year_data, season)
    all_data.append(year_data)

fetching 1995-1996 season data...
107 dates
fetching 1996-1997 season data...
109 dates
fetching 1997-1998 season data...
106 dates
fetching 1998-1999 season data...
105 dates
fetching 1999-2000 season data...
105 dates
fetching 2000-2001 season data...
98 dates
fetching 2001-2002 season data...
103 dates
fetching 2002-2003 season data...
98 dates
fetching 2003-2004 season data...
98 dates
fetching 2004-2005 season data...
100 dates
fetching 2005-2006 season data...
106 dates
fetching 2006-2007 season data...
96 dates
fetching 2007-2008 season data...
97 dates
fetching 2008-2009 season data...
99 dates
fetching 2009-2010 season data...
107 dates
fetching 2010-2011 season data...
106 dates
fetching 2011-2012 season data...
105 dates
fetching 2012-2013 season data...
102 dates
fetching 2013-2014 season data...
96 dates
fetching 2014-2015 season data...
97 dates


In [116]:
# load data from json files

all_data = []

for year in range(1995, 2015):
    fname = '../data/{}-{}.json'.format(year, year + 1)
    with open(fname) as f:
        season_data = json.load(f)
        all_data.append(season_data)

print '{} seasons of data loaded!'.format(len(all_data))

20 seasons of data loaded!


In [133]:
# transpose data to one row per season, team, and week

data_formatted = []

for season in all_data:
    season_id = season[0]['season']    
    results_by_team = defaultdict(lambda : defaultdict(dict))
    
    for week in season:
        for team in week['standings']:
            results_by_team[team['name']][team['games_played']] = team
    
    for team, results in results_by_team.iteritems():
        for games_played, details in results.iteritems():
            entry = {
                'season': season_id,
                'team': team,
                'games': games_played,
                'points': details['points'],
                'goals_diff': details['goals_diff'],
                'goals_total': details['goals_total'],
            }
            data_formatted.append(entry)

In [176]:
# make pandas df from newly formatted data

df = pd.read_json(json.dumps(data_formatted))

cols = ['season', 'team', 'games', 'points', 'goals_diff', 'goals_total']
df = df[cols]

print len(df)
print df.head()
print df.tail()

15288
      season                 team  games  points  goals_diff  goals_total
0  1995-1996  Queens Park Rangers     24      18         -18           18
1  1995-1996  Queens Park Rangers     25      18         -20           18
2  1995-1996  Queens Park Rangers     26      18         -21           19
3  1995-1996  Queens Park Rangers     27      21         -19           22
4  1995-1996  Queens Park Rangers     20      18         -14           15

[5 rows x 6 columns]
          season             team  games  points  goals_diff  goals_total
15283  2014-2015  West Ham United     36      47           0           43
15284  2014-2015  West Ham United     35      47           1           43
15285  2014-2015  West Ham United     34      44           0           42
15286  2014-2015  West Ham United     33      43           0           42
15287  2014-2015  West Ham United     32      43           2           42

[5 rows x 6 columns]


In [190]:
# sort by points, goals_diff, goals_total

dg = df.copy()

dg = dg[dg.games > 0]

dg.sort(
    ['season', 'games', 'points', 'goals_diff', 'goals_total'], 
    ascending=[False, True, False, False, False], 
    inplace=True
)

print dg.head()

          season             team  games  points  goals_diff  goals_total
15183  2014-2015          Chelsea      1       3           2            3
15068  2014-2015  Manchester City      1       3           2            2
14532  2014-2015     Swansea City      1       3           1            2
14647  2014-2015        Liverpool      1       3           1            2
14839  2014-2015          Arsenal      1       3           1            2

[5 rows x 6 columns]


In [197]:
# add rank column

def add_rank(group):
    pts_lead = max(group.points)
    rk, pts, gd, gt = (0,) * 4
    rk_increment = 1
    for i, row in group.iterrows():
        if row.points != pts or row.goals_diff != gd or row.goals_total != gt:
            rk += rk_increment
            rk_increment = 1
        else:
            rk_increment += 1
        group.loc[i, 'rank'] = rk
        group.loc[i, 'pts_from_lead'] = pts_lead - row.points
        pts, gd, gt = row.points, row.goals_diff, row.goals_total
    return group

dg = dg.groupby(['season', 'games']).apply(add_rank)

new_cols = ['rank', 'pts_from_lead']
dg[new_cols] = dg[new_cols].astype(int)

dg.head(10)

,season,team,games,points,goals_diff,goals_total,rank,pts_from_lead
15183,2014-2015,Chelsea,1,3,2,3,1,0
15068,2014-2015,Manchester City,1,3,2,2,2,0
14532,2014-2015,Swansea City,1,3,1,2,3,0
14647,2014-2015,Liverpool,1,3,1,2,3,0
14839,2014-2015,Arsenal,1,3,1,2,3,0
14686,2014-2015,Tottenham Hotspur,1,3,1,1,6,0
14801,2014-2015,Aston Villa,1,3,1,1,6,0
14916,2014-2015,Hull City,1,3,1,1,6,0
14992,2014-2015,West Bromwich Albion,1,1,0,2,9,2
15030,2014-2015,Sunderland,1,1,0,2,9,2


In [198]:
# final tables by season

final_tables = dg[dg.games == 38]

final_tables = final_tables[['season', 'team', 'rank']]

final_tables.rename(columns={'rank': 'final_rank'}, inplace=True)

print final_tables.head()

          season               team  final_rank
15193  2014-2015            Chelsea           1
15078  2014-2015    Manchester City           2
14848  2014-2015            Arsenal           3
14618  2014-2015  Manchester United           4
14695  2014-2015  Tottenham Hotspur           5

[5 rows x 3 columns]


In [199]:
# join week by week standings with final standings
dh = pd.merge(dg, final_tables, on=['season', 'team'])

dh.head()

,season,team,games,points,goals_diff,goals_total,rank,pts_from_lead,final_rank
0,2014-2015,Chelsea,1,3,2,3,1,0,1
1,2014-2015,Chelsea,2,6,4,5,2,0,1
2,2014-2015,Chelsea,3,9,7,11,1,0,1
3,2014-2015,Chelsea,4,12,9,15,1,0,1
4,2014-2015,Chelsea,5,13,9,16,1,0,1


In [201]:
# save df as tsv

dh.to_csv('../output/weekly_standings.tsv', sep='\t', index=False)